In [1]:
# 装饰模式的应用场景:
#　让其函数在不需要做任何代码变动的情况下增加额外的功能，装饰器的返回值也是一个函数对象，经常用于有插入需求的场景
#　比如插入日志，性能测试，食物处理，缓存，权限校验等场景，有了装饰器，就可以抽离出大量与函数功能无关的雷同代码继续重用
# ref
# https://www.cnblogs.com/cicaday/p/python-decorator.html

In [2]:
#　装饰器实现的场景模拟
#　inspect可以获取父函数的方法作为额存储，inspect.stack()[1][3]
# 但是这个有一个缺陷是，每次都要在新增的函数里面要加入这个装饰器的方法，显的比较笨拙，而且这种方法比较受到限

In [11]:
def debug():
    import inspect
    caller_name = inspect.stack()[1][3]
    print("[DEBUG]:enter {}()".format(caller_name))

In [12]:
def say_hello():
    debug()
    print("hello")

In [13]:
def say_goodbye():
    debug()
    print("goodbye")

In [14]:
if __name__ == "__main__":
    say_hello()
    say_goodbye()

[DEBUG]:enter say_hello()
hello
[DEBUG]:enter say_goodbye()
goodbye


In [16]:
# 早期的装饰器版本，是通过一个warapper的方法将函数包裹起来，直接调用这个包裹的函数，实现装饰的作用，实际上称为包裹器还更靠谱一点
# 实际上返回的是一个函数形式
#　实现的逐步调用的方法
#　step one 返回装饰器需要表达的含义，基func.__name__或者func的相关方法
#　step two return func的相关方法
#　step three:返回wrapper的方法

In [23]:
def debug(func):
    def wrapper():
        print("[DUBUG]: ENTER {}()".format(func.__name__))
        return func() # 获取func的函数输出
    return wrapper()　#　这样则直接返回的是函数的结果

In [24]:
def say_hello():
    print("hello")

In [25]:
say_hello = debug(say_hello) # 实际上将函数重新包裹了一下，和装饰器这个概念还是有一点不自然，耦合的比较紧

[DUBUG]: ENTER say_hello()
hello


In [22]:
# 在后续版本中出现了增加@语法糖的方法，这种方法实现了和需要装饰的函数的剥离，是一种相对更好的方法
# 这种方法更简单，只需要在需要被装饰的函数前加上@即可,利用这个实现了再另外包裹一层的目的

In [36]:
def debug(func):
    def wrapper():
        print("[DUBUG]: ENTER {}()".format(func.__name__)) # 获取装饰器输出
        return func() #获取函数的输出
    return wrapper # 返回的是函数，后面需要做二次调用，　这种情况下是否加()则不影响

In [37]:
@debug
def say_hello():
    print("hello")

In [35]:
say_hello()

[DUBUG]: ENTER say_hello()
hello


In [38]:
#在上面装饰器的实现方法中，并未提到在装饰函数中添加参数的情况，如果需要添加参数，在wrapper中添加即可
# wrapper实现对参数的传递
#　wrapper() takes 0 positional arguments but 1 was given
#　不然会提示错误，如果wrapper不提前将func所需的参数加入进来

In [50]:
def debug(func):
    def wrapper(somthing):
        print("[DEBUG]: ENTER {} ()".format(func.__name__))
        return func(somthing)
    return wrapper

In [51]:
@debug
def say_hello(something):
    print( "hello {}".format(something))

In [52]:
say_hello("Lee")

[DEBUG]: ENTER say_hello ()
hello Lee


In [53]:
#　同时这个存在的另外一个问题，一旦参数限定，就会出现debug和say_hello紧耦合了
#　所以需要将参数变成不定的形式，这样引入不定参数的想法是合理的
#　*args/kwargs的使用就顺理成章了

In [75]:
def debug(func):
    def wrapper(*args):
        print("[DEBUG]: ENTER {}()".format(func.__name__))
        return func(*args, **kwargs)
    return wrapper

In [76]:
@debug
# 可以有无穷的参数输入，在这里args是tuple,但是*args映射的为其中的值，不再是tuple的形式, kwargs也是这种含义
def say(something, Lee, Loo):
    print(type(something))
    print(type(Lee))
    print("hello {}, {}, {}".format(something, Lee, Loo))

In [77]:
say("Lee", "Jack", "Loo")

[DEBUG]: ENTER say()
<class 'str'>
<class 'str'>
hello Lee, Jack, Loo


In [78]:
#　装饰器进阶
#　带参数的装饰器和类装饰器

In [79]:
# 带参数的装饰器
#　则直接在装饰器上加入参数,也即在上面的装饰器上再包一层

In [80]:
def logging(level):
    def wrapper(func):
        def inner_wrapper(*args, **kwargs):
            print("[{level}]: enter function {func}()".format( level=level, func = func.__name__))
            return func(*args, *kwargs)
        return inner_wrapper
    return wrapper

In [81]:
@logging(level="INFO")
def say(something):
    print("hello {}".format(something))

In [82]:
say("Lee")

[INFO]: enter function say()
hello Lee


In [83]:
@logging(level="Debug")
def say(something):
    print("hello {}".format(something))

In [84]:
say("Lee")

[Debug]: enter function say()
hello Lee


In [ ]:
# 基于类的实现的装饰器
#　需要一个__call__的方法

In [86]:
class Test():
    def __call__(self):
        print("call me")

In [87]:
# 基于类做装饰器, 这种情况是基于__call__函数可以直接访问的优点，其实调用方法和非类的调用方法并无差异

In [100]:
class logging(object):
    def __init__(self, func):
        self.func = func
    def __call__(self, *args, **kwargs):
        print("[DEBUG] enter function {func}".format(func = self.func.__name__))
        return self.func(*args, **kwargs)

In [101]:
@logging
def say(something):
    print("say {}".format(something))

In [102]:
say("Lee")

[DEBUG] enter function say
say Lee


In [103]:
#　带参数的类装饰器，只需要在类的初始化里将参数加进来，然后在call函数里调用
# 这个不需要要加入初始化方法

In [116]:
class logging(object):
    def __init__(self, func, level="INFO"):
        self.level = level
        self.func = func
    def __call__(self, *args, **kwargs):
        print("[{level}] enter function {func}".format(level=self.level, func=self.func.__name__))
        return self.func(*args, **kwargs)

In [117]:
@logging
def say(something):
    print("say {}".format(something))

In [118]:
say("Lee")

[INFO] enter function say
say Lee


In [119]:
# 另外一种形式

In [138]:
class logging(object):
    def __init__(self, level="INFO"):
        self.level = level
    def __call__(self, func):
        def wrapper(*args, **kwargs):
            print("[{level}] enter function {func}".format(level=self.level, func=func.__name__))
            func(*args, **kwargs)
        return wrapper

In [143]:
#　这个需要加入初始化方法
@logging()
def say(something):
    print("say {}".format(something))

In [144]:
say("Lee")

[INFO] enter function say
say Lee
